In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Sun/ML Shock/Final project/

/content/drive/MyDrive/Sun/ML Shock/Final project


In [ ]:
import sys
# sys.path.insert(0,"C:/Users/Amy/Desktop/Green_Git/eegClassification/utils")
sys.path.insert(0,"/content/drive/MyDrive/Sun/ML Shock/Final project/")

In [ ]:
# Import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from CustomDataset import CustomDataset

In [ ]:
# Load (train or test) data from csv file
# path = "C:/Users/Amy/Desktop/Green_Git/eegClassification/sample_data/"
# path_df = "C:/Users/Amy/Desktop/Green_Git/eegClassification/files/"
# scaler_path = "C:/Users/Amy/Desktop/Green_Git/eegClassification/models/scalers/"

path = "./data/"
path_df = "./"
scaler_path = "./data/"
path_model = "./model_custom_v1.pt"


In [ ]:
# Type of input data
data_type = "spec" # "eeg_raw" #"eeg_spec"  #
# Type of mscale
scaler_type = "min_max" #"standard" #
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 64

In [ ]:
if data_type == "spec":
    input_shape = (4,299,100)
elif data_type == "eeg_spec":
    input_shape = (20,129,43)
elif data_type == "eeg_raw":
    input_shape = (20,9800)

In [ ]:
def load_data(path, path_df, data_type, text):

    train=True
    df = pd.read_csv(path_df + f"{text}_processed.csv")



    data_dir = (
        f"train_eegs/"
        if "eeg" in data_type
        else f"train_spectrograms/"
    )
    data_dir = path + data_dir

    ###
    # files = os.listdir(data_dir)
    # df = df[
    #     df["eeg_id" if "eeg" in data_type else "spectrogram_id"].isin(
    #         [int(f.split(".")[0]) for f in files]
    #     )
    # ]
    ###

    votes_cols = [
        "seizure_vote",
        "lpd_vote",
        "gpd_vote",
        "lrda_vote",
        "grda_vote",
        "other_vote",
    ]
    label_cols = (
        ["eeg_id", "label_id", "eeg_label_offset_seconds"]
        if "eeg" in data_type
        else ["spectrogram_id", "label_id", "spectrogram_label_offset_seconds"]
    )
    offset = (
        ["eeg_label_offset_seconds"]
        if "eeg" in data_type
        else ["spectrogram_label_offset_seconds"]
    )

    # if info_cols not in df add it and set to zero
    for col in offset:
        if col not in df.columns:
            df[col] = 0
    # if df does not contain "label_id" add a unique label_id
    if "label_id" not in df.columns:
        df["label_id"] = range(len(df))

    info = {}
    df_gr = df.groupby(label_cols)
    for name, group in df_gr:
        # first row of group
        info[name] = {"votes": group[votes_cols].values[0] if train else None}

    return data_dir, data_type, info


In [ ]:
data_dir, data_type, info = load_data(path, path_df, data_type, "train")
data_dir, data_type, len(info)

('./data/train_spectrograms/', 'spec', 65509)

In [ ]:
train_data = CustomDataset(data_dir, data_type, info)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)


In [ ]:
data_dir, data_type, info_val  = load_data(path, path_df, data_type, "val")
data_dir, data_type, len(info_val)

('./data/train_spectrograms/', 'spec', 21777)

In [ ]:
valid_data = CustomDataset(data_dir, data_type, info_val)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)

In [ ]:
data_dir, data_type, info_test = load_data(path, path_df, data_type, "test")
data_dir, data_type, len(info_test)

In [ ]:
test_data = CustomDataset(data_dir, data_type, info_test)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)

In [ ]:
# loop through data and save it as a numpy array
count = 0
for X, label, votes  in train_loader:
    # save the images
    np.save(f"./data_prep/train/images_{count}.npy", X.numpy())
    # save the labels
    np.save(f"./data_prep/train/labels_{count}.npy", label.numpy())
    # save the votes
    np.save(f"./data_prep/train/votes_{count}.npy", votes.numpy())

    count += 1

    if count > 100:
        break
    


In [ ]:
# loop through data and save it as a numpy array
count = 0
for X, label, votes  in valid_loader:
    # save the images
    np.save(f"./data_prep/valid/images_{count}.npy", X.numpy())
    # save the labels
    np.save(f"./data_prep/valid/labels_{count}.npy", label.numpy())
    # save the votes
    np.save(f"./data_prep/valid/votes_{count}.npy", votes.numpy())

    count += 1

    if count > 34:
        break
    

In [ ]:
# loop through data and save it as a numpy array
count = 0
for X, label, votes  in test_loader:
    # save the images
    np.save(f"./data_prep/test/images_{count}.npy", X.numpy())
    # save the labels
    np.save(f"./data_prep/test/labels_{count}.npy", label.numpy())
    # save the votes
    np.save(f"./data_prep/test/votes_{count}.npy", votes.numpy())

    count += 1

    if count > 34:
        break
    